<a href="https://colab.research.google.com/github/shanikairoshi/QFL-with-DUN/blob/main/Qiskit_VQC_FL_Fashion_MNIST_Experiemental_Anlysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit
!pip install qiskit.aer
#Install required Dependencies
!pip install --upgrade seaborn
!pip install --upgrade scikit-learn
!pip install --upgrade matplotlib
!pip install --upgrade pandas
!pip install --upgrade qiskit
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from qiskit import *
import numpy as np
from qiskit.utils import algorithm_globals
!pip install qiskit_machine_learning

import time
# installing TensorFLow Version 2.3.1
from IPython.display import clear_output

clear_output()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 30.0 MB/s eta 0:00:00


In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit import Parameter
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

from qiskit.utils import algorithm_globals
import numpy as np
#from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit.library import ZZFeatureMap, TwoLocal, RealAmplitudes
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
#from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_machine_learning.algorithms.classifiers import VQC
#from qiskit_machine_learning.optimizers import SPSA
from qiskit import BasicAer
from qiskit import BasicAer, execute
from qiskit.algorithms.optimizers import SPSA


Data Preprocessing

In [ ]:
#importing the Fashion MNIST dataset from keras
from tensorflow.keras.datasets import fashion_mnist as dataset
(X_train, y_train), (X_test, y_test) = dataset.load_data()
# Load the FeMNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
import pandas as pd
from tensorflow.keras.datasets import fashion_mnist

# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Reshape the data if needed (e.g., flattening the images)
# x_train = x_train.reshape(x_train.shape[0], -1)
# x_test = x_test.reshape(x_test.shape[0], -1)

# Create DataFrames
train_df = pd.DataFrame({'label': y_train})
test_df = pd.DataFrame({'label': y_test})

# Add pixel values to DataFrames
for i in range(x_train.shape[1] * x_train.shape[2]):
    train_df[f'pixel_{i}'] = x_train[:, i // x_train.shape[2], i % x_train.shape[2]]
    test_df[f'pixel_{i}'] = x_test[:, i // x_test.shape[2], i % x_test.shape[2]]

# Save DataFrames to CSV
train_df.to_csv('fashion_mnist_train.csv', index=False)
test_df.to_csv('fashion_mnist_test.csv', index=False)

clear_output()

In [ ]:
image_size = 28 # width and length are equal
data_path= "/content/"
train_data = np.loadtxt(data_path + "fashion_mnist_train.csv", delimiter=",",skiprows=1)
test_data = np.loadtxt(data_path + "fashion_mnist_test.csv", delimiter=",",skiprows=1)

In [ ]:
img = train_data[:, 1:][1].reshape((image_size, image_size))
plt.imshow(img)
plt.show()

In [ ]:
#Extracting features and labels from the dataset and truncating the dataset to 10,000 datapoints
train_data_features = train_data[:10000, 1:]
train_data_labels = train_data[:10000, :1].reshape(10000,)

# Using SVD to reduce dimensions to 10
tsvd = TruncatedSVD(n_components=10)
X_SVD = tsvd.fit_transform(train_data_features)

# Use t-SNE technique to reduce dimensions to 2
np.random.seed(0)
tsne = TSNE(n_components=2)
train_data_features_reduced = tsne.fit_transform(X_SVD)

In [ ]:
# function to help plot the 2-D dataset
def plotdataset(X, Y, c1, c2, N):
    lbl1 = f'Component {c1}'
    lbl2 = f'Component {c2}'
    df = pd.DataFrame({lbl1:X[:N,c1], lbl2:X[:N,c2], 'label':Y[:N]})
    sns.lmplot(data=df, x=lbl1, y=lbl2, fit_reg=False, hue='label', scatter_kws={'alpha':0.5})
plotdataset(train_data_features_reduced, train_data_labels, 0, 1, N=2000)

In [ ]:
zero_datapoints_array = [] #an array of the data points containing value 0
one_datapoints_array = []# an array of the data points containing value 1
for i in range(10000):
    if train_data_labels[i] == 0:                   # extracting zeros
        zero_datapoints_array.append(train_data_features_reduced[i])

for i in range(10000):
    if train_data_labels[i] == 1:                   # extracting ones
        one_datapoints_array.append(train_data_features_reduced[i])

zero_datapoints_array = np.array(zero_datapoints_array)
one_datapoints_array = np.array(one_datapoints_array)

def normalize(arr, max_val, n):
    a = np.divide(arr, max_val)
    return a + n
zero_datapoints_normalized = normalize(zero_datapoints_array, 100, 1)
one_datapoints_normalized = normalize(one_datapoints_array, 100, 1)

In [ ]:

train_size = 200
test_size = 50
dp_size_zero = 5
dp_size_one = 5

zero_train = zero_datapoints_normalized[:train_size]
one_train = one_datapoints_normalized[:train_size]

zero_test = zero_datapoints_normalized[train_size + 1:train_size + test_size + 1]
one_test = one_datapoints_normalized[train_size + 1:train_size + test_size + 1]

training_input = {'A':zero_train, 'B':one_train}
test_input = {'A':zero_test, 'B':one_test}

# datapoints is our validation set
datapoints = []
dp_zero = zero_datapoints_normalized[train_size + test_size + 2:train_size + test_size + 2 + dp_size_zero]
dp_one = one_datapoints_normalized[train_size + test_size + 2:train_size + test_size + 2 + dp_size_one]
datapoints.append(np.concatenate((dp_zero, dp_one)))
dp_y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
datapoints.append(dp_y)

class_to_label = {'A': 0, 'B': 1}

In [ ]:
import numpy as np

# Assuming you have the data defined as per your initial code
train_size = 200
test_size = 50
dp_size_zero = 5
dp_size_one = 5

zero_train = zero_datapoints_normalized[:train_size]
one_train = one_datapoints_normalized[:train_size]

zero_test = zero_datapoints_normalized[train_size + 1:train_size + test_size + 1]
one_test = one_datapoints_normalized[train_size + 1:train_size + test_size + 1]

training_input = {'A': zero_train, 'B': one_train}
test_input = {'A': zero_test, 'B': one_test}

datapoints = []
dp_zero = zero_datapoints_normalized[train_size + test_size + 2:train_size + test_size + 2 + dp_size_zero]
dp_one = one_datapoints_normalized[train_size + test_size + 2:train_size + test_size + 2 + dp_size_one]
datapoints.append(np.concatenate((dp_zero, dp_one)))
dp_y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
datapoints.append(dp_y)

class_to_label = {'A': 0, 'B': 1}

# Split data for 5 clients
num_clients = 5
client_data = {f'Client_{i+1}': {'TRAIN_DATA': [], 'TRAIN_LABELS': [], 'TEST_DATA': [], 'TEST_LABELS': []} for i in range(num_clients)}

for i in range(num_clients):
    start_idx = i * (train_size // num_clients)
    end_idx = (i + 1) * (train_size // num_clients)

    client_data[f'Client_{i+1}']['TRAIN_DATA'] = np.concatenate((zero_train[start_idx:end_idx], one_train[start_idx:end_idx]))
    client_data[f'Client_{i+1}']['TRAIN_LABELS'] = np.array([class_to_label['A']] * (end_idx - start_idx) + [class_to_label['B']] * (end_idx - start_idx))

    start_idx = i * (test_size // num_clients)
    end_idx = (i + 1) * (test_size // num_clients)

    client_data[f'Client_{i+1}']['TEST_DATA'] = np.concatenate((zero_test[start_idx:end_idx], one_test[start_idx:end_idx]))
    client_data[f'Client_{i+1}']['TEST_LABELS'] = np.array([class_to_label['A']] * (end_idx - start_idx) + [class_to_label['B']] * (end_idx - start_idx))

# Print client data
#for client, data in client_data.items():
    #print(f'{client}:')
    #print(data)
    #print()


test code below

In [ ]:
# Set random seed
algorithm_globals.random_seed = 3142
np.random.seed(algorithm_globals.random_seed)

from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
feature_dim = 2
pauli_feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=1, paulis = ['Z','X','ZY'])
pauli_feature_map.draw()

# Define the feature map and variational form
#FEATURE_MAP = pauli_feature_map
#FEATURE_MAP = ZZFeatureMap(feature_dimension=2, reps=2)
FEATURE_MAP=ZFeatureMap(feature_dimension=2, reps=2)

VAR_FORM = TwoLocal(2, ['ry', 'rz'], 'cz', reps=2)

# Combine feature map and variational form to create the circuit
AD_HOC_CIRCUIT = FEATURE_MAP.compose(VAR_FORM)
AD_HOC_CIRCUIT.measure_all()
AD_HOC_CIRCUIT.decompose().draw()

class localOptimizerLog:
    """Log to store optimizer's intermediate results"""
    def __init__(self):
        self.evaluations = []
        self.parameters = []
        self.costs = []
    def update(self, evaluation, parameter, cost, _stepsize, _accept):
        """Save intermediate results. Optimizer passes five values
        but we ignore the last two."""
        self.evaluations.append(evaluation)
        self.parameters.append(parameter)
        self.costs.append(cost)

initial_point = np.random.random(VAR_FORM.num_parameters)

#Global optimizer log
class GlobalOptimizerLog:
    """Log to store optimizer's intermediate results"""
    def __init__(self):
        self.evaluations = []
        self.parameters = []
        self.costs = []
    def update(self, evaluation, parameter, cost, _stepsize, _accept):
        """Save intermediate results. Optimizer passes five values
        but we ignore the last two."""
        self.evaluations.append(evaluation)
        self.parameters.append(parameter)
        self.costs.append(cost)

global_optimizer_log = GlobalOptimizerLog()

# Initialize global model
global_model = VQC(
    feature_map=FEATURE_MAP,
    ansatz=VAR_FORM,
    loss='cross_entropy',
    optimizer=SPSA(callback=global_optimizer_log.update),
    initial_point=initial_point,
    quantum_instance=BasicAer.get_backend('qasm_simulator')
)


#Local Training with Global Training
import warnings
warnings.filterwarnings("ignore")
# Training settings
num_rounds = 2
learning_rate = 0.01  # Adjust as needed

# Create a list of VQC instances, one for each client
clients = []
# Initialize an empty list to store client logs
client_logs = []
# Initialize an empty list to store client parameters
client_parameters = []
global_parameters=[]
global_cost=[]

# Array to store global cost
GlobalCostperRoundAll=[]

# Initialize global parameters
global_parameters = np.random.rand(VAR_FORM.num_parameters)  # Initialize with random values or any desired initial values

# Federated training loop
for round_num in range(num_rounds):
    global_cost=[]
    client_logs=[]
    aggregated_params = np.zeros_like(global_parameters)  # Initialize aggregated parameters
    # Train each client's data on their VQC models
    for client_id, data in client_data.items():
        train_data = data['TRAIN_DATA']
        train_labels = data['TRAIN_LABELS']
        test_data = data['TEST_DATA']
        test_labels = data['TEST_LABELS']
        #train_data, train_labels, test_data, test_labels= data

        # Set up the optimization
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")  # Suppress warnings
        Locallog = localOptimizerLog()
        optimizer = SPSA(maxiter=100, callback=Locallog.update)
        vqc = VQC(feature_map=FEATURE_MAP,
          ansatz=VAR_FORM,
          loss='cross_entropy',
          optimizer=SPSA(callback=Locallog.update),
          initial_point=global_parameters,
          quantum_instance=BasicAer.get_backend('qasm_simulator'))

        vqc.fit(train_data, train_labels)
        clients.append(vqc)
        client_logs.append(Locallog)
        client_parameters.append(Locallog.parameters[-1])
        cost = Locallog.costs[-1]
        print(f"Round {round_num}, Client {client_id}: parameters = {Locallog.parameters[-1]}")


    #aggregated_params = np.zeros_like(global_parameters)  # Initialize aggregated parameters
    aggregated_params = np.mean(client_parameters, axis=0)  # Calculate mean of parameters

    # Update global parameters for the next round
    global_parameters = aggregated_params
    # Aggregate client parameters
    print(f"Round {round_num},global: aggregated_gradients = {global_parameters}")

    # Update the global optimizer log with the aggregated parameters
    global_optimizer_log.parameters.append(global_parameters)
    global_model.fit(train_data, train_labels)
    global_cost = global_optimizer_log.costs
    print(f"Round {round_num},global cost: {global_cost}")

print("Federated training done")

import csv as csv
# Slice the data if needed (e.g., taking the last 100 elements)
data_to_save =global_optimizer_log.costs

# Define the CSV file path
csv_file_path = "global_optimizer_costsZfeatureMap.csv"

# Write the data to the CSV file
with open(csv_file_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Cost"])  # Write a header row if needed
    for cost in data_to_save:
        writer.writerow([cost])

print(f"Data saved to {csv_file_path}")

In [ ]:
#csv_file_pathPauliCost = "global_optimizer_costsPaulifeatureMap.csv"
#csv_file_pathZZCost = "global_optimizer_costsZZfeatureMap.csv"
csv_file_pathZCost = "global_optimizer_costsZfeatureMap.csv"

# Initialize an empty list to store the data

PauliCost = []
ZZCost=[]
ZCost=[]

print(len(ZCost))


#================================================
# Read data from the CSV file and append it to the list


#===============================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZCost, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZCost.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZCost}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#each client cost variations over epochs for whole set code
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 4))
# Plot cost for each client over rounds
epoch = len(client_logs[0].evaluations)
for client_id, log in enumerate(client_logs):
    plt.plot(range(epoch), log.costs, label=f'Client {client_id + 1}', linestyle='dashed')
#plt.plot(range(epoch), ZZCost[-100:], label="ZZFeatureMap", linewidth=2,color='orange')#LR=0.01, maxiter=100
#plt.plot(range(epoch), ZCost[-100:], label="global-SPSA-500", linewidth=1, marker='*',color='blue')#LR=0.01 maxiter=500
#plt.plot(range(epoch), PauliCost[-100:], label="PauliFeatureMap", linewidth=2, color='red')
#plt.plot(range(epoch), global_optimizer_log.costs, label='Global Cost')
plt.plot(range(epoch), ZCost[-100:], label="ZFeatureMap", linewidth=2, color='green')
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.title('QiskitFL Cost Evolution Different FeatureMaps-TWOLocal Circuit-FeMnist Data')
plt.legend()
plt.show()

Actual code below

In [ ]:
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap

def CircuitStruWithFeatureMap(CircuitStructure, FeatureMap, FileName):
  # Set random seed
  algorithm_globals.random_seed = 3142
  np.random.seed(algorithm_globals.random_seed)


  #VAR_FORM = TwoLocal(2, ['ry', 'rz'], 'cz', reps=2)
  FEATURE_MAP=FeatureMap
  VAR_FORM=CircuitStructure

  # Combine feature map and variational form to create the circuit
  AD_HOC_CIRCUIT = FEATURE_MAP.compose(VAR_FORM)
  AD_HOC_CIRCUIT.measure_all()
  AD_HOC_CIRCUIT.decompose().draw()

  class localOptimizerLog:
    """Log to store optimizer's intermediate results"""
    def __init__(self):
        self.evaluations = []
        self.parameters = []
        self.costs = []
    def update(self, evaluation, parameter, cost, _stepsize, _accept):
        """Save intermediate results. Optimizer passes five values
        but we ignore the last two."""
        self.evaluations.append(evaluation)
        self.parameters.append(parameter)
        self.costs.append(cost)

  initial_point = np.random.random(VAR_FORM.num_parameters)

#Global optimizer log
  class GlobalOptimizerLog:
    """Log to store optimizer's intermediate results"""
    def __init__(self):
        self.evaluations = []
        self.parameters = []
        self.costs = []
    def update(self, evaluation, parameter, cost, _stepsize, _accept):
        """Save intermediate results. Optimizer passes five values
        but we ignore the last two."""
        self.evaluations.append(evaluation)
        self.parameters.append(parameter)
        self.costs.append(cost)

  global_optimizer_log = GlobalOptimizerLog()

  # Initialize global model
  global_model = VQC(
    feature_map=FEATURE_MAP,
    ansatz=VAR_FORM,
    loss='cross_entropy',
    optimizer=SPSA(callback=global_optimizer_log.update),
    initial_point=initial_point,
    quantum_instance=BasicAer.get_backend('qasm_simulator')
  )


  #Local Training with Global Training
  import warnings
  warnings.filterwarnings("ignore")
  # Training settings
  num_rounds = 2
  learning_rate = 0.01  # Adjust as needed

  # Create a list of VQC instances, one for each client
  clients = []
  # Initialize an empty list to store client logs
  client_logs = []
  # Initialize an empty list to store client parameters
  client_parameters = []
  global_parameters=[]
  global_cost=[]

  # Array to store global cost
  GlobalCostperRoundAll=[]

  # Initialize global parameters
  global_parameters = np.random.rand(VAR_FORM.num_parameters)  # Initialize with random values or any desired initial values

  # Federated training loop
  for round_num in range(num_rounds):
      global_cost=[]
      client_logs=[]
      aggregated_params = np.zeros_like(global_parameters)  # Initialize aggregated parameters
      # Train each client's data on their VQC models
      for client_id, data in client_data.items():
          train_data = data['TRAIN_DATA']
          train_labels = data['TRAIN_LABELS']
          test_data = data['TEST_DATA']
          test_labels = data['TEST_LABELS']
          #train_data, train_labels, test_data, test_labels= data

          # Set up the optimization
          with warnings.catch_warnings():
              warnings.simplefilter("ignore")  # Suppress warnings
          Locallog = localOptimizerLog()
          optimizer = SPSA(maxiter=100, callback=Locallog.update)
          vqc = VQC(feature_map=FEATURE_MAP,
            ansatz=VAR_FORM,
            loss='cross_entropy',
            optimizer=SPSA(callback=Locallog.update),
            initial_point=global_parameters,
            quantum_instance=BasicAer.get_backend('qasm_simulator'))

          vqc.fit(train_data, train_labels)
          clients.append(vqc)
          client_logs.append(Locallog)
          client_parameters.append(Locallog.parameters[-1])
          cost = Locallog.costs[-1]
          print(f"Round {round_num}, Client {client_id}: parameters = {Locallog.parameters[-1]}")


      #aggregated_params = np.zeros_like(global_parameters)  # Initialize aggregated parameters
      aggregated_params = np.mean(client_parameters, axis=0)  # Calculate mean of parameters

      # Update global parameters for the next round
      global_parameters = aggregated_params
      # Aggregate client parameters
      print(f"Round {round_num},global: aggregated_gradients = {global_parameters}")

      # Update the global optimizer log with the aggregated parameters
      global_optimizer_log.parameters.append(global_parameters)
      global_model.fit(train_data, train_labels)
      global_cost = global_optimizer_log.costs
      print(f"Round {round_num},global cost: {global_cost}")
      global_model.score(test_data,test_labels)

  print("Federated training done")

  import csv as csv
  # Slice the data if needed (e.g., taking the last 100 elements)
  data_to_save =global_optimizer_log.costs

  # Define the CSV file path
  csv_file_path = FileName

  # Write the data to the CSV file
  with open(csv_file_path, mode='w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      writer.writerow(["Cost"])  # Write a header row if needed
      for cost in data_to_save:
          writer.writerow([cost])

  print(f"Data saved to {csv_file_path}")



In [ ]:
#Test circuit structures with Z fm
from qiskit.circuit.library import RealAmplitudes,EfficientSU2
import csv as csv
feature_dim = 2
# Define the feature map and variational form

feature_dim = 2
num_qubits=2

FEATURE_MAPZ=ZFeatureMap(feature_dimension=2, reps=2)

#Test 01
variational_circ1 = TwoLocal(2, ['ry', 'rz'], 'cz', reps=2)
fileName1="TwoLocalwithZ.csv"

variational_circ2 = RealAmplitudes(num_qubits, entanglement='full', reps=3)
fileName2="RealAmplitudeswithZ.csv"

variational_circ3=EfficientSU2(feature_dim, reps=2)
fileName3="EfficientSU2withz.csv"


CircuitStruWithFeatureMap(variational_circ1,FEATURE_MAPZ,fileName1)
print("Test 01- TwoLocalwithZ completed")
print("---------")
CircuitStruWithFeatureMap(variational_circ2,FEATURE_MAPZ,fileName2)
print("Test 02- RealAmplitudeswithZ completed")
print("---------")
CircuitStruWithFeatureMap(variational_circ3,FEATURE_MAPZ,fileName3)
print("Test 03- EfficientSU2withPauli completed")
print("---------")

# Initialize an empty list to store the data

TwoLocalwithZ = []
RealAmplitudeswithZ=[]
EfficientSU2withz=[]


# Read data from the CSV file and append it to the list
try:
    with open(fileName1, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            TwoLocalwithZ.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{fileName1}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#================================================
# Read data from the CSV file and append it to the list
try:
    with open(fileName2, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            RealAmplitudeswithZ.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{fileName2}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

#===============================================
# Read data from the CSV file and append it to the list
try:
    with open(fileName3, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            EfficientSU2withz.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{fileName3}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#each client cost variations over epochs for whole set code
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 4))
# Plot cost for each client over rounds
epoch = 100
#for client_id, log in enumerate(client_logs):
    #plt.plot(range(epoch), log.costs, label=f'Client {client_id + 1}', linestyle='dashed')
plt.plot(range(epoch), RealAmplitudeswithZ[-100:], label="RealAmplitudes", linewidth=2,color='orange')#LR=0.01, maxiter=100
#plt.plot(range(epoch), ZCost[-100:], label="global-SPSA-500", linewidth=1, marker='*',color='blue')#LR=0.01 maxiter=500
plt.plot(range(epoch), TwoLocalwithZ[-100:], label="TwoLocal", linewidth=2, color='red')
#plt.plot(range(epoch), global_optimizer_log.costs, label='Global Cost')
plt.plot(range(epoch), EfficientSU2withz[-100:], label="EfficientSU2", linewidth=2, color='green')
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.title('QiskitFL TWOLocal Circuit Structure-FeMnist Data', fontsize=10)
plt.legend()
plt.show()


In [ ]:
#each client cost variations over epochs for whole set code
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 4))
# Plot cost for each client over rounds
epoch = 100
#for client_id, log in enumerate(client_logs):
    #plt.plot(range(epoch), log.costs, label=f'Client {client_id + 1}', linestyle='dashed')
plt.plot(range(epoch), TwoLocalwithZ[-100:], label="TwoLocal", linewidth=1, color='blue')

plt.plot(range(epoch), RealAmplitudeswithZ[-100:], label="RealAmplitudes", linewidth=1,color='red')#LR=0.01, maxiter=100
#plt.plot(range(epoch), ZCost[-100:], label="global-SPSA-500", linewidth=1, marker='*',color='blue')#LR=0.01 maxiter=500
#plt.plot(range(epoch), global_optimizer_log.costs, label='Global Cost')
plt.plot(range(epoch), EfficientSU2withz[-100:], label="EfficientSU2", linewidth=1 ,color='green')
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.title('QiskitFL TWOLocal Circuit Structure-FeMnist Data', fontsize=10)
plt.legend()
plt.show()

In [ ]:
from qiskit.circuit.library import RealAmplitudes,EfficientSU2
import csv as csv
feature_dim = 2
# Define the feature map and variational form

feature_dim = 2
FEATURE_MAPpauli = PauliFeatureMap(feature_dimension=feature_dim, reps=1, paulis = ['Z','X','ZY'])
#pauli_feature_map.draw()
FEATURE_MAPZZ = ZZFeatureMap(feature_dimension=2, reps=2)
FEATURE_MAPZ=ZFeatureMap(feature_dimension=2, reps=2)

#Test 01
variational_circ1 = TwoLocal(2, ['ry', 'rz'], 'cz', reps=2)
fileNamePauli1="TwoLocalwithPauli.csv"
fileNameZ1="TwoLocalwithZ.csv"
fileNameZZ1='TwoLocalwithZZ.csv'


CircuitStruWithFeatureMap(variational_circ1,FEATURE_MAPZ,fileNameZ1)
print("Test 01- part 1 z feature map completed")
print("---------")
CircuitStruWithFeatureMap(variational_circ1,FEATURE_MAPZZ,fileNameZZ1)
print("Test 01- part 1 zz feature map completed")
print("---------")
CircuitStruWithFeatureMap(variational_circ1,FEATURE_MAPpauli,fileNamePauli1)
print("Test 01- part 1 pauli feature map completed")
print("---------")



In [ ]:

csv_file_pathPauliCost = "TwoLocalwithPauli.csv"
csv_file_pathZZCost = "TwoLocalwithZZ.csv"
csv_file_pathZCost = "TwoLocalwithZ.csv"

# Initialize an empty list to store the data

PauliCost = []
ZZCost=[]
ZCost=[]

print(len(ZCost))

# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathPauliCost, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            PauliCost.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathPauliCost}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#================================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZZCost, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZZCost.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZZCost}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

#===============================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZCost, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZCost.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZCost}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#each client cost variations over epochs for whole set code
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 4))
# Plot cost for each client over rounds
epoch = 100
#for client_id, log in enumerate(client_logs):
    #plt.plot(range(epoch), log.costs, label=f'Client {client_id + 1}', linestyle='dashed')
plt.plot(range(epoch), ZZCost[-100:], label="ZZFeatureMap", linewidth=2,color='orange')#LR=0.01, maxiter=100
#plt.plot(range(epoch), ZCost[-100:], label="global-SPSA-500", linewidth=1, marker='*',color='blue')#LR=0.01 maxiter=500
plt.plot(range(epoch), PauliCost[-100:], label="PauliFeatureMap", linewidth=2, color='red')
#plt.plot(range(epoch), global_optimizer_log.costs, label='Global Cost')
plt.plot(range(epoch), ZCost[-100:], label="ZFeatureMap", linewidth=2, color='green')
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.title('QiskitFL TWOLocal Circuit Structure-FeMnist Data', fontsize=10)
plt.legend()
plt.show()


In [ ]:


#Test 02
num_qubits=2
variational_circ2 = RealAmplitudes(num_qubits, entanglement='full', reps=3)
fileNamePauli2="RealAmplitudeswithPauli.csv"
fileNameZ2="RealAmplitudeswithZ.csv"
fileNameZZ2='RealAmplitudeswithZZ.csv'

CircuitStruWithFeatureMap(variational_circ2,FEATURE_MAPZ,fileNameZ2)
print("Test 02- part 1 z feature map completed")
print("---------")
CircuitStruWithFeatureMap(variational_circ2,FEATURE_MAPZZ,fileNameZZ2)
print("Test 02- part 1 zz feature map completed")
print("---------")
CircuitStruWithFeatureMap(variational_circ2,FEATURE_MAPpauli,fileNamePauli2)
print("Test 02- part 1 pauli feature map completed")
print("---------")



In [ ]:
csv_file_pathPauliCost = "RealAmplitudeswithPauli.csv"
csv_file_pathZZCost = "RealAmplitudeswithZZ.csv"
csv_file_pathZCost = "RealAmplitudeswithZ.csv"

# Initialize an empty list to store the data

PauliCost = []
ZZCost=[]
ZCost=[]

# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathPauliCost, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            PauliCost.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathPauliCost}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#================================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZZCost, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZZCost.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZZCost}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

#===============================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZCost, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZCost.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZCost}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#each client cost variations over epochs for whole set code
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 4))
# Plot cost for each client over rounds
epoch = 100
#for client_id, log in enumerate(client_logs):
    #plt.plot(range(epoch), log.costs, label=f'Client {client_id + 1}', linestyle='dashed')
plt.plot(range(epoch), ZZCost[-100:], label="ZZFeatureMap", linewidth=2,color='orange')#LR=0.01, maxiter=100
#plt.plot(range(epoch), ZCost[-100:], label="global-SPSA-500", linewidth=1, marker='*',color='blue')#LR=0.01 maxiter=500
plt.plot(range(epoch), PauliCost[-100:], label="PauliFeatureMap", linewidth=2, color='red')
#plt.plot(range(epoch), global_optimizer_log.costs, label='Global Cost')
plt.plot(range(epoch), ZCost[-100:], label="ZFeatureMap", linewidth=2, color='green')
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.title('QiskitFL RealiAmplitude Circuit Structure-FeMnist Data', fontsize=10)
plt.legend()
plt.show()

In [ ]:

#Test 03
variational_circ3=EfficientSU2(feature_dim, reps=2)
fileNamePauli3="EfficientSU2withPauli.csv"
fileNameZ3="EfficientSU2withZ.csv"
fileNameZZ3='EfficientSU2withZZ.csv'

CircuitStruWithFeatureMap(variational_circ3,FEATURE_MAPZ,fileNameZ3)
print("Test 03- part 1 z feature map completed")
print("---------")
CircuitStruWithFeatureMap(variational_circ3,FEATURE_MAPZZ,fileNameZZ3)
print("Test 03- part 1 zz feature map completed")
print("---------")
CircuitStruWithFeatureMap(variational_circ3,FEATURE_MAPpauli,fileNamePauli3)
print("Test 03- part 1 pauli feature map completed")
print("---------")



In [ ]:
csv_file_pathPauliCost = "EfficientSU2withPauli.csv"
csv_file_pathZZCost = "EfficientSU2withZZ.csv"
csv_file_pathZCost = "EfficientSU2withZ.csv"

# Initialize an empty list to store the data

PauliCost = []
ZZCost=[]
ZCost=[]

print(len(ZCost))

# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathPauliCost, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            PauliCost.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathPauliCost}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#================================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZZCost, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZZCost.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZZCost}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

#===============================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZCost, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZCost.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZCost}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#each client cost variations over epochs for whole set code
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 4))
# Plot cost for each client over rounds
epoch = 100
#for client_id, log in enumerate(client_logs):
    #plt.plot(range(epoch), log.costs, label=f'Client {client_id + 1}', linestyle='dashed')
plt.plot(range(epoch), ZZCost[-100:], label="ZZFeatureMap", linewidth=2,color='orange')#LR=0.01, maxiter=100
#plt.plot(range(epoch), ZCost[-100:], label="global-SPSA-500", linewidth=1, marker='*',color='blue')#LR=0.01 maxiter=500
plt.plot(range(epoch), PauliCost[-100:], label="PauliFeatureMap", linewidth=2, color='red')
#plt.plot(range(epoch), global_optimizer_log.costs, label='Global Cost')
plt.plot(range(epoch), ZCost[-100:], label="ZFeatureMap", linewidth=2, color='green')
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.title('QiskitFL RealiAmplitude Circuit Structure-FeMnist Data', fontsize=10)
plt.legend()
plt.show()

In [ ]:


csv_file_pathZTwoLocal = "TwoLocalwithZ.csv"
csv_file_pathZReal = "RealAmplitudeswithZ.csv"
csv_file_pathZEff = "EfficientSU2withZ.csv"

csv_file_pathPauliTwoLocal =  "TwoLocalwithPauli.csv"
csv_file_pathPauliReal = "RealAmplitudeswithPauli.csv"
csv_file_pathPauliEff ="EfficientSU2withPauli.csv"

csv_file_pathZZTwoLocal = "TwoLocalwithZZ.csv"
csv_file_pathZZReal = "RealAmplitudeswithZZ.csv"
csv_file_pathZZEff = "EfficientSU2withZZ.csv"

# Initialize an empty list to store the data
# Initialize an empty list to store the data

ZTwoLocal = []
ZReal=[] #ZZCost
ZEff=[] #Zcost

PauliTwoLocal = []
PauliReal=[] #ZZCost
PauliEff=[] #Zcost

ZZTwoLocal = []
ZZReal=[] #ZZCost
ZZEff=[] #Zcost

#***************************ZZfeatureMap*******************************
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZTwoLocal, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZTwoLocal.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZTwoLocal}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#================================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZReal, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZReal.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZReal}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

#===============================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZEff, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZEff.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZEff}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")
#==================================================================


#***************************PaulifeatureMap*******************************

# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathPauliTwoLocal, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            PauliTwoLocal.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathPauliTwoLocal}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#================================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathPauliReal, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            PauliReal.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathPauliReal}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

#===============================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathPauliEff, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            PauliEff.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathPauliEff}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#***************************ZZfeatureMap*******************************

try:
    with open(csv_file_pathZZTwoLocal, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZZTwoLocal.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZZTwoLocal}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


#================================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZZReal, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZZReal.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZZReal}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

#===============================================
# Read data from the CSV file and append it to the list
try:
    with open(csv_file_pathZZEff, mode='r', newline='') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row if present
        for row in reader:
            # Assuming the CSV file has one column
            ZZEff.append(float(row[0]))  # Convert to float if needed
except FileNotFoundError:
    print(f"The file '{csv_file_pathZZEff}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")
#==================================================================

#each client cost variations over epochs for whole set code
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 4))
# Plot cost for each client over rounds
epoch = 100
#for client_id, log in enumerate(client_logs):
    #plt.plot(range(epoch), log.costs, label=f'Client {client_id + 1}', linestyle='dashed')

plt.plot(range(epoch), ZReal[-100:], label="Z_with_Real ", linewidth=2,color='blue',linestyle='dashed')#LR=0.01, maxiter=100
#plt.plot(range(epoch), ZCost[-100:], label="global-SPSA-500", linewidth=1, marker='*',color='blue')#LR=0.01 maxiter=500
plt.plot(range(epoch), ZTwoLocal[-100:], label="Z_with_Two", linewidth=2, color='green')
#plt.plot(range(epoch), global_optimizer_log.costs, label='Global Cost')
plt.plot(range(epoch), ZEff[-100:], label="Z_with_Eff", linewidth=2, color='cyan',linestyle='dotted')


plt.plot(range(epoch), PauliReal[-100:], label="Pauli_with_Real ", linewidth=2,color='orange',linestyle='dashed')#LR=0.01, maxiter=100
#plt.plot(range(epoch), ZCost[-100:], label="global-SPSA-500", linewidth=1, marker='*',color='blue')#LR=0.01 maxiter=500
plt.plot(range(epoch), PauliTwoLocal[-100:], label="Pauli_with_Two", linewidth=2, color='red')
#plt.plot(range(epoch), global_optimizer_log.costs, label='Global Cost')
plt.plot(range(epoch), PauliEff[-100:], label="Pauli_with_Eff", linewidth=2, color='yellow',linestyle='dotted')



plt.plot(range(epoch), ZZReal[-100:], label="ZZ_with_Real", linewidth=2,color='black',linestyle='dashed')#LR=0.01, maxiter=100
#plt.plot(range(epoch), ZCost[-100:], label="global-SPSA-500", linewidth=1, marker='*',color='blue')#LR=0.01 maxiter=500
plt.plot(range(epoch), ZZTwoLocal[-100:], label="ZZ_with_Two", linewidth=2, color='purple')
#plt.plot(range(epoch), global_optimizer_log.costs, label='Global Cost')
plt.plot(range(epoch), ZZEff[-100:], label="ZZ_with_Eff", linewidth=2, color='brown',linestyle='dotted')

plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.title('QiskitFL Different Circuit Structures and FeatureMaps-FeMnist Data', fontsize=10)
plt.legend()
plt.show()

### ***********************************Implement Femnist with Qiskit VQC****************


In [ ]:



def filter_data(x, y):
  """
  Helper Function to filter the dataset
  x is image independent feature , y is dependent feature
  """
  #filter the data using labels
  keep = (y == 5) | (y == 9)
  #value for keep vaiable is true only for y=lable 5 and lable 9
  x, y = x[keep], y[keep]

  # convert labels to boolean
  # y = True if y==5
  # y = False if y==9
  y = y == 5
  return x,y

#===========================

#importing the Fashion MNIST dataset from keras
from tensorflow.keras.datasets import fashion_mnist as dataset
(X_train, y_train), (X_test, y_test) = dataset.load_data()
print("The shape of the X_train is {}".format( X_train.shape))
print("The shape of the y_train is {}".format(y_train.shape))
print("The shape of the X_test is {}".format(X_test.shape))
print("The shape of the y_test is {}".format(y_test.shape))


#Filter the train set
X_train, y_train = filter_data(X_train, y_train)

#Filter the test_set
X_test, y_test = filter_data(X_test, y_test)

# Let's have a look at the shapes of train and test data
print("           ")
print("=======Filter data for binary classification")
print("The shape of the X_train is {}".format( X_train.shape))
print("The shape of the y_train is {}".format(y_train.shape))
print("The shape of the X_test is {}".format(X_test.shape))
print("The shape of the y_test is {}".format(y_test.shape))

#Let's have a look at the first image from our X_train and the
# corresponding label from y_train
print("")
print("The First Image has the label {}".format(y_train[0]))
plt.imshow(X_train[0])
plt.colorbar()
plt.title('Visualization of the Dataset')
plt.show()

#Normalizing the train and test image data
X_train = X_train/255.0
X_test = X_test/ 255.0

#Let's again have a look at the first image from our X_train and
#see if we have successfully normalized the datasets
plt.imshow(X_train[0])
plt.colorbar()
print("The First Image after pixel normalization")
plt.title('Visualization of the Dataset')
plt.show()


#Before proceeding, we need to reshape our images in the dataset
x_train= X_train.reshape(X_train.shape[0], *(28,28,1))#X_train.shape[0] indicate number of images in xtrain, 1 indicate channels here since grayscale 1
x_test= X_test.reshape(X_test.shape[0], *(28,28,1))


# Downscaling the images
X_train= tf.image.resize(x_train, (2,2)).numpy()
X_test= tf.image.resize(x_test, (2,2)).numpy()

#Let's again have a look at the first image from our resized X_train
plt.imshow(X_train[0,:,:,0])
plt.colorbar()
plt.title('Visualization of the Resized Dataset')
plt.show()


#Splitting the training fdataset into train and validation datasets
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.15, random_state=0)

print("The shape of the X_train is {}".format(X_train.shape))
print("The shape of the y_train is {}".format(y_train.shape))
print("The shape of the X_valid is {}".format(X_valid.shape))
print("The shape of the y_valid is {}".format(y_valid.shape))

In [ ]:


# Normalize the pixel values to range [0, 1]
#x_train = x_train.astype('float32') / 255.0
#x_test = x_test.astype('float32') / 255.0

# Flatten the images (convert 28x28 images to 1D arrays of length 784)
#x_train_flattened = x_train.reshape(x_train.shape[0], -1)
#x_test_flattened = x_test.reshape(x_test.shape[0], -1)

#======================================================================
#Federated Data preprocess

# Split the data among clients
n_clients = 5  # Set the number of clients (adjust as needed)
client_data = []

# Split the training data among clients using train_test_split
for i in range(n_clients):
    x_train_client, _, y_train_client, _ = train_test_split(
        x_train, y_train, test_size=0.9 / n_clients, random_state=i
    )
    client_data.append((x_train_client, y_train_client))

# Print the shapes of each client's dataset
for i, (x_train_client, y_train_client) in enumerate(client_data):
    print(f"Client {i+1} - x_train shape: {x_train_client.shape}, y_train shape: {y_train_client.shape}")

# Load the MNIST dataset
#(_, y_train), (_, y_test) = mnist.load_data()

# Convert the class labels to one-hot encoded format
num_classes = 2 # Number of classes in the MNIST dataset
#y_train_onehot = tf.one_hot(y_train, num_classes)
#y_test_onehot = tf.one_hot(y_test, num_classes)

# Print the shape of one-hot encoded labels
#print("Shape of y_train_onehot:", y_train_onehot.shape)  # (60000, 10)
#print("Shape of y_test_onehot:", y_test_onehot.shape)

In [ ]:
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap

def CircuitStruWithFeatureMap(CircuitStructure, FeatureMap, FileName):
  # Set random seed
  algorithm_globals.random_seed = 3142
  np.random.seed(algorithm_globals.random_seed)


  #VAR_FORM = TwoLocal(2, ['ry', 'rz'], 'cz', reps=2)
  FEATURE_MAP=FeatureMap
  VAR_FORM=CircuitStructure

  # Combine feature map and variational form to create the circuit
  AD_HOC_CIRCUIT = FEATURE_MAP.compose(VAR_FORM)
  AD_HOC_CIRCUIT.measure_all()
  AD_HOC_CIRCUIT.decompose().draw()

  class localOptimizerLog:
    """Log to store optimizer's intermediate results"""
    def __init__(self):
        self.evaluations = []
        self.parameters = []
        self.costs = []
    def update(self, evaluation, parameter, cost, _stepsize, _accept):
        """Save intermediate results. Optimizer passes five values
        but we ignore the last two."""
        self.evaluations.append(evaluation)
        self.parameters.append(parameter)
        self.costs.append(cost)

  initial_point = np.random.random(VAR_FORM.num_parameters)

#Global optimizer log
  class GlobalOptimizerLog:
    """Log to store optimizer's intermediate results"""
    def __init__(self):
        self.evaluations = []
        self.parameters = []
        self.costs = []
    def update(self, evaluation, parameter, cost, _stepsize, _accept):
        """Save intermediate results. Optimizer passes five values
        but we ignore the last two."""
        self.evaluations.append(evaluation)
        self.parameters.append(parameter)
        self.costs.append(cost)

  global_optimizer_log = GlobalOptimizerLog()

  # Initialize global model
  global_model = VQC(
    feature_map=FEATURE_MAP,
    ansatz=VAR_FORM,
    loss='cross_entropy',
    optimizer=SPSA(callback=global_optimizer_log.update),
    initial_point=initial_point,
    quantum_instance=BasicAer.get_backend('qasm_simulator')
  )


  #Local Training with Global Training
  import warnings
  warnings.filterwarnings("ignore")
  # Training settings
  num_rounds = 50
  learning_rate = 0.01  # Adjust as needed

  # Create a list of VQC instances, one for each client
  clients = []
  # Initialize an empty list to store client logs
  client_logs = []
  # Initialize an empty list to store client parameters
  client_parameters = []
  global_parameters=[]
  global_cost=[]

  # Array to store global cost
  GlobalCostperRoundAll=[]

  # Initialize global parameters
  global_parameters = np.random.rand(VAR_FORM.num_parameters)  # Initialize with random values or any desired initial values

  # Federated training loop
  for round_num in range(num_rounds):
      global_cost=[]
      client_logs=[]
      aggregated_params = np.zeros_like(global_parameters)  # Initialize aggregated parameters
      # Train each client's data on their VQC models
      for client_id, data in client_data.items():
          train_data = data['TRAIN_DATA']
          train_labels = data['TRAIN_LABELS']
          test_data = data['TEST_DATA']
          test_labels = data['TEST_LABELS']
          #train_data, train_labels, test_data, test_labels= data

          # Set up the optimization
          with warnings.catch_warnings():
              warnings.simplefilter("ignore")  # Suppress warnings
          Locallog = localOptimizerLog()
          optimizer = SPSA(maxiter=100, callback=Locallog.update)
          vqc = VQC(feature_map=FEATURE_MAP,
            ansatz=VAR_FORM,
            loss='cross_entropy',
            optimizer=SPSA(callback=Locallog.update),
            initial_point=global_parameters,
            quantum_instance=BasicAer.get_backend('qasm_simulator'))

          vqc.fit(train_data, train_labels)
          clients.append(vqc)
          client_logs.append(Locallog)
          client_parameters.append(Locallog.parameters[-1])
          cost = Locallog.costs[-1]
          print(f"Round {round_num}, Client {client_id}: parameters = {Locallog.parameters[-1]}")


      #aggregated_params = np.zeros_like(global_parameters)  # Initialize aggregated parameters
      aggregated_params = np.mean(client_parameters, axis=0)  # Calculate mean of parameters

      # Update global parameters for the next round
      global_parameters = aggregated_params
      # Aggregate client parameters
      print(f"Round {round_num},global: aggregated_gradients = {global_parameters}")

      # Update the global optimizer log with the aggregated parameters
      global_optimizer_log.parameters.append(global_parameters)
      global_model.fit(train_data, train_labels)
      global_cost = global_optimizer_log.costs
      print(f"Round {round_num},global cost: {global_cost}")

  print("Federated training done")

  import csv as csv
  # Slice the data if needed (e.g., taking the last 100 elements)
  data_to_save =global_optimizer_log.costs

  # Define the CSV file path
  csv_file_path = FileName

  # Write the data to the CSV file
  with open(csv_file_path, mode='w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      writer.writerow(["Cost"])  # Write a header row if needed
      for cost in data_to_save:
          writer.writerow([cost])

  print(f"Data saved to {csv_file_path}")



In [ ]:
# FLattening the images
X_train = X_train.reshape(X_train.shape[0], *(1,4,1))
X_valid = X_valid.reshape(X_valid.shape[0], *(1,4,1))
X_test = X_test.reshape(X_test.shape[0], *(1,4,1))

#Let's have a look on the first example
print(X_train[0])



In [ ]:

def binary_encode(X,threshold=0.5):
  """
  Encodes the given datset to use binary encoding

  Parameters:
  X(array) : Image data to be processed for encoding
  threshold(float): Threshold for binary encoding, 0.5 by default

  Returns:
  encoded_images(array): Binary encoded Image Data

  """
  encoded_images = list()
  for image in X:
    # pixel value is 1 if it's greater than threshold or else zero
    encoded_image = [1 if j>threshold else 0 for j in image[0]]
    encoded_images.append(encoded_image)
  return np.array(encoded_images)




In [ ]:
 X_train = binary_encode(X_train)
print(X_train[0])

In [ ]:
from qiskit.circuit.library import ZZFeatureMap
circuit = ZZFeatureMap(3, reps=1, insert_barriers=True)
circuit.decompose().draw()
x = [0.1, 0.2, 0.3]
encode = circuit.bind_parameters(x)
encode.decompose().draw()

Data encoding

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_bloch_multivector

# Load MNIST data (replace this with actual MNIST data loading code)
# For this example, let's consider a single grayscale image with 4x4 pixels
mnist_image = np.array([
    [0.2, 0.5, 0.8, 0.7],
    [0.4, 0.6, 0.3, 0.1],
    [0.9, 0.3, 0.2, 0.5],
    [0.6, 0.4, 0.7, 0.2]
])

# Quantum circuit parameters
n_qubits = 16  # Number of qubits (should match the image dimensions)
state_vector_simulator = Aer.get_backend('statevector_simulator')

# Normalize pixel values and convert to binary strings
binary_strings = []
for row in mnist_image:
    for pixel in row:
        binary_strings.append(format(int(pixel * 255), '08b'))

# Create a quantum circuit
qc = QuantumCircuit(n_qubits)

# Apply amplitude encoding
for i, binary_string in enumerate(binary_strings):
    for j, bit in enumerate(binary_string):
        qubit_index = i * 4 + j
        if qubit_index < n_qubits:
            if bit == '1':
                qc.x(qubit_index)  # Apply an X gate to set the qubit to |1>

# Simulate the quantum state
job = assemble(transpile(qc, state_vector_simulator))
result = state_vector_simulator.run(job).result()
statevector = result.get_statevector()

# Print and visualize the statevector
print("Encoded Quantum Statevector:")
print(statevector)

# Visualize the Bloch sphere representation of the statevector
#plot_bloch_multivector(statevector)


In [ ]:

# Define the QNN architecture
n_qubits = 8  # Number of qubits in the quantum circuit
n_layers = 2  # Number of layers in the quantum circuit

# Create a quantum circuit with trainable parameters
def create_qnn_circuit(params):
    qc = QuantumCircuit(n_qubits)

    # Apply trainable quantum gates for each layer
    for layer in range(n_layers):
        for qubit in range(n_qubits):
            # Add trainable parameters to the quantum gates
            theta = params[layer * n_qubits + qubit]
            qc.rx(theta, qubit)

        # Apply CNOT gate to entangle the qubits
        for qubit in range(n_qubits - 1):
            qc.cx(qubit, qubit + 1)

    return qc

# Initialize random parameters for each client's QNN
n_params = n_qubits * n_layers
np.random.seed(0)  # Set a seed for reproducibility
client_params = [np.random.rand(n_params) for _ in range(len(client_datasets))]


# Simulate quantum circuit execution and calculate loss
# Simulate quantum circuit execution and calculate loss
# Simulate quantum circuit execution and calculate loss
def simulate_qnn(params, data, true_labels):
    # Create a quantum circuit for the QNN
    circuit = create_qnn_circuit(params)

    # Simulator backend
    backend = Aer.get_backend('statevector_simulator')

    # Execute the quantum circuit
    job = execute(circuit, backend)
    result = job.result()
    statevector = result.get_statevector()

    # Reshape the statevector to match the shape of data
    statevector = statevector.reshape(-1, statevector.shape[-1])

    # Calculate the prediction probabilities from statevector
    prediction = np.abs(statevector) ** 2

    print(prediction.shape)
    print(true_labels.shape)

    # Ensure true_labels and prediction have the same shape
    if true_labels.shape != prediction.shape:
        raise ValueError("Shapes of true_labels and prediction do not match.")

    # Calculate the loss (cross-entropy between target and prediction)
    loss = -np.mean(np.sum(true_labels * np.log(prediction), axis=1))

    return loss

# Federated learning loop
n_iterations = 10
learning_rate = 0.1

for iteration in range(n_iterations):
    # Perform training on each client's dataset
    for i, data in enumerate(client_datasets):
        client_data, client_labels = data
        true_labels_onehot = tf.one_hot(client_labels, num_classes)
        loss = simulate_qnn(client_params[i], data, true_labels_onehot)

        # Calculate gradient and update parameters (simple gradient descent)
        gradients = np.gradient(client_params[i], loss)
        client_params[i] = client_params[i] - learning_rate * gradients

    # Aggregate parameters from all clients and calculate global parameters
    global_params = np.mean(client_params, axis=0)

# Example usage:
# Assuming you have the client_datasets and true_labels arrays from the previous code
# And 'global_params' from the federated learning loop

# Let's simulate QNN for the first client dataset:
client_1_data, client_1_labels = client_datasets[0]
loss_client_1 = simulate_qnn(global_params, client_1_data, tf.one_hot(client_1_labels, num_classes))
print(f"Loss for client 1: {loss_client_1}")


